Ερώτημα 2

In [8]:
import pandas as pd

from pyomo.environ import *
from pyomo.gdp import *

Δημιουργούμε ένα python dictionary "tasks" στον οποίο αποθηκεύουμε τις 8 διεργασίες που πρέπει να πραγματοποιήσουμε την διάρκεια της καθεμίας, αλλά και τις διεργασίες που πρέπει να προηγηθούν

In [9]:
tasks = {
    ('tap1','Blue')   : {'dur': 45, 'previous': None},
    ('tap1','Yellow') : {'dur': 10, 'previous': ('tap1','Blue')},
    ('tap2','Blue')   : {'dur': 20, 'previous': ('tap2','Green')},
    ('tap2','Green')  : {'dur': 10, 'previous': None},
    ('tap2','Yellow') : {'dur': 34, 'previous': ('tap2','Blue')},
    ('tap3','Blue')   : {'dur': 12, 'previous': ('tap3','Yellow')},
    ('tap3','Green')  : {'dur': 17, 'previous': ('tap3','Blue')},
    ('tap3','Yellow') : {'dur': 28, 'previous': None},   
}


Ορίζουμε την συνάρτηση tapModel η οποία ουσιαστικά, παίνει ως όρισμα το tasks και το μετατρέπει στο μοντέλο μας

In [10]:
def tapModel(tasks):
    
    #Δημιουργούμε το μοντέλο
    model = ConcreteModel()

    # Κάνουμε το tasks έναν διδιάστατο πίνακα, παίρνοντας τα στοιχεία από το tasks
    model.tasks = Set(initialize = tasks.keys(), dimen=2)
    
    # Με την βοήθεια ενός της set βάζουμε τις εργασίες σε ένα python set
    model.jobs = Set(initialize = list(set([j for (j,m) in model.tasks])))
    
    # Επίσης σε ένα python set βάζουμε και το σετ των μηχανημάτων
    model.machines = Set(initialize = list(set([m for (j,m) in model.tasks])))
    
    # Παίρνοντας τα στοιχεία από το τασκ και φιλτράροντας τα, δημιουργούμε έναν τετραδιάστατο πίνακα με την σωστή σειρά,
    # της κάθε εργασίας, κάθε ταπετσαρίας, σε κάθε μηχάνημα. Θα μας χρειαστούν για τους conjunctive περιορισμούς.
    model.correctOrder = Set(initialize = model.tasks * model.tasks, dimen=4, 
        filter = lambda model, j, m, k, n: (k,n) == tasks[(j,m)]['previous'])
    
    # Δημιουργούμε έναν πίνακα λαμβάνοντας υπόψη την ταπετσαρία, την διεργασία και την μηχανή, που θα χρησιμοποιηθεί
    # όταν θέτουμε τους disjunctive περιορισμούς
    model.disjuncs = Set(initialize = model.jobs * model.jobs * model.machines, dimen=3,
        filter = lambda model, j, k, m: j < k and (j,m) in model.tasks and (k,m) in model.tasks)
    
    # Φορτώνουμε την διάρκεια της κάθε εργασίας σαν παράμετρους.
    model.dur = Param(model.tasks, initialize=lambda model, j, m: tasks[(j,m)]['dur'])

    # Θέτουμε το Μ όσο το άθροισμα όλων των διαρκειων για να είναι αρκετά μεγάλο ώστε να μην επηρεάσει το αποτέλεσμα, 
    # αλλά όχι και υπερβολικά μεγάλο, ώστε να μπορούσε να δημιουργήσει κάποιο υπολογιστικό πρόβλημα.
    # Το μεγάλο Μ χρησιμοποιείται και ως ανώτατο όριο για το finish time
    M = sum([model.dur[j, m] for (j,m) in model.tasks])
    
    # Δημιουργούμε τα decision variables και τους βάζουμε ως άνω όριο το μεγάλο Μ για να αποφύγουμε υπολογιστικά προβλήματα.
    model.totalTime = Var(bounds=(0, M))
    model.start = Var(model.tasks, bounds=(0, M))
    
    # Θέτουμε τον στόχο του μοντέλου που είναι να ελαχιστοποιηθεί ο συνολικός χρόνος
    model.objective = Objective(expr = model.totalTime, sense = minimize)

    # Θέτουμε τους περιορισμούς που δηλώνουν ότι ο συνολικός χρόνος πρέπει να είναι μεγαλύτερος από τον χρόνο που τελειώνει 
    # η κάθε διεργασία
    model.finish = Constraint(model.tasks, rule=lambda model, j, m:  
        model.start[j,m] + model.dur[j,m] <= model.totalTime)
    
    # Θέτουμε τους conjunctive περιορισμούς μας, που έχουν να κάνουν δηλαδή με την σωστή προτεραιότητα για κάθε ταπετσαρία
    # χρησιμοποιώντας τον φιλτραρισμένο τετραδιάστατο πίνακα που φτιάξαμε προηγουμένως
    model.preceding = Constraint(model.correctOrder, rule=lambda model, j, m, k, n: 
        model.start[k,n] + model.dur[k,n] <= model.start[j,m])
    
    # Θέτουμε τους disjunctive περιορισμούς, δηλαδή ότι κάθε μηχανή μπορεί να διεκπεραιώσει μόνη μια εργασία την φορά.
    model.disjunctions = Disjunction(model.disjuncs, rule=lambda model,j,k,m:
        [model.start[j,m] + model.dur[j,m] <= model.start[k,m], 
         model.start[k,m] + model.dur[k,m] <= model.start[j,m]])
    
    #Για να χρησιμοποιήσουμε τον glpk solver πρέπει πρώτα να μετατρέψουμε το μοντέλο μας σε standard MILP/MINLP model.
    TransformationFactory('gdp.bigm').apply_to(model)
    return model

tapModel(tasks)


Ορίζουμε μια συνάρτηση που παίρνει σαν όρισμα το μοντέλο μας, και επιστρέφει για κάθε διεργασία, την ταπετσαρία, το μηχάνημα, τον χρόνο που ξεκίνησε, την διάρκεια και τον χρόνο που τελείωσε. Για την επίλυση χρησιμοποιούμε το glpk solver.

In [12]:
def tapSolve(model):
    SolverFactory('glpk').solve(model)
    results = [{'Job': j,
                'Machine': m,
                'Start': model.start[j, m](), 
                'Duration': model.dur[j,m], 
                'Finish': model.start[(j, m)]() + model.dur[j,m]}
               for j,m in model.tasks]
    return results

Καλούμε την TapSolve για το μοντέλο μας και εκτυπώνουμε τα αποτελέσματα

In [13]:
results = tapSolve(tapModel(tasks))
results

[{'Job': 'tap1',
  'Machine': 'Blue',
  'Start': 42.0,
  'Duration': 45,
  'Finish': 87.0},
 {'Job': 'tap1',
  'Machine': 'Yellow',
  'Start': 87.0,
  'Duration': 10,
  'Finish': 97.0},
 {'Job': 'tap2',
  'Machine': 'Blue',
  'Start': 10.0,
  'Duration': 20,
  'Finish': 30.0},
 {'Job': 'tap2',
  'Machine': 'Green',
  'Start': 0.0,
  'Duration': 10,
  'Finish': 10.0},
 {'Job': 'tap2',
  'Machine': 'Yellow',
  'Start': 53.0,
  'Duration': 34,
  'Finish': 87.0},
 {'Job': 'tap3',
  'Machine': 'Blue',
  'Start': 30.0,
  'Duration': 12,
  'Finish': 42.0},
 {'Job': 'tap3',
  'Machine': 'Green',
  'Start': 42.0,
  'Duration': 17,
  'Finish': 59.0},
 {'Job': 'tap3',
  'Machine': 'Yellow',
  'Start': 0.0,
  'Duration': 28,
  'Finish': 28.0}]

Βάζουμε σε έναν πίνακα τα αποτελέσματα για να είναι πιο ευανάγνωστα

In [14]:
schedule = pd.DataFrame(results)

print('\nSchedule by Tapetsaria')
print(schedule.sort_values(by=['Job','Start']).set_index(['Job', 'Machine']))



Schedule by Tapetsaria
              Start  Duration  Finish
Job  Machine                         
tap1 Blue      42.0        45    87.0
     Yellow    87.0        10    97.0
tap2 Green      0.0        10    10.0
     Blue      10.0        20    30.0
     Yellow    53.0        34    87.0
tap3 Yellow     0.0        28    28.0
     Blue      30.0        12    42.0
     Green     42.0        17    59.0


Συνεπώς, ο μικρότερος δυνατός χρόνος είναι 97 λεπτά και αυτός προκύπτει άν:
Ταπετσαρία 1: περιμένουμε μέχρι το 42' την βάλουμε στην μπλε μηχανή και με το που τελειώσει (87')  την βάλουμε κατευθείαν στην κίτρινη μηχανή.
Ταπετσαρία 2: Μπαίνει κατευθείαν (0') στην πράσινη μηχανή, και αμέσως όταν τελειώσει (10') στην μπλε, όταν βγει από την μπλε (30') περιμένουμε μέχρι το 53' και την τοποθετούμε στην κίτρινη.
Ταπετσαρία 3: Μπαίνει κατευθείαν (0') στην κίτρινη μηχανή, και όταν τελειώσει (28') περιμένουμε μέχρι το 30' για να την βάλουμε στην μπλε. Όταν βγει απο την μπλέ (42') μπαίνει κατευθείαν στην πράσινη.